# Expoloitation de donnnées

In [22]:
import pandas as pd

## DATA

### Capacité d'accueil

https://www.capareseau.fr/#

In [23]:
current_file = '2501_CapacitesDAccueil'
root = '/Users/matthiasmolenat/repos/congestion/data/'
path_capa_accueil = f'{root}{current_file}.csv'
path_corrected_capa_accueil = f'{root}corrected_{current_file}.csv'

In [24]:
with open(path_capa_accueil, 'r', encoding='utf-8') as file:
    content = file.read()
    
modified_content = content.replace('RH�NE', 'RHONE')
modified_content = modified_content.replace('k�', 'k€')
modified_content = modified_content.replace('�?', 'é') 
modified_content = modified_content.replace('e?', 'é') 
modified_content = modified_content.replace('d�un', "d'un") 
modified_content = modified_content.replace(' � ', "à") 
modified_content = modified_content.replace('�', "é") 

# Write the modified content to a new CSV file
with open(path_corrected_capa_accueil, 'w', encoding='utf-8') as file:
    file.write(modified_content)

In [25]:
df = pd.read_csv(path_corrected_capa_accueil, sep=';')

In [26]:
# sous-tirage 
df["ratio_congestion_sous_tirage"] = 5*df["Consommation minimale"]/df["Puissance cumulée des transformateurs existants"]*100
df = df[df["ratio_congestion_sous_tirage"]<110]
df = df[~df['ratio_congestion_sous_tirage'].isna()]
df["ratio_congestion_sous_tirage"] = df["ratio_congestion_sous_tirage"].astype(int)
df["RAM_sous_tirage"] = (1 -  df["ratio_congestion_sous_tirage"]/100)*df["Puissance cumulée des transformateurs existants"]


# injection 
df["projet_injection"] = df['Puissance EnR déjé raccordée'] + df["Puissance des projets EnR en développement"] + df["Puissance en file d'attente hors S3REnR majorée de la capacité réservée du S3REnR"]
df["ratio_congestion_injection"] = df["projet_injection"]  / df["Capacité de transformation HTB/HTA restante disponible pour l'injection sur le réseau public de distribution"]*100
df["ratio_congestion_injection"] = [100 if x > 100 else x for x in df["ratio_congestion_injection"].to_list()] 
df = df[~df['ratio_congestion_injection'].isna()]
df["ratio_congestion_injection"] = df["ratio_congestion_injection"].astype(int)
df["RAM_injection"] = (1-df["ratio_congestion_injection"]/100)*df["Capacité de transformation HTB/HTA restante disponible pour l'injection sur le réseau public de distribution"]

# intersection
df['min_RAM'] = [min(df['RAM_sous_tirage'].iloc[i],df['RAM_injection'].iloc[i]) for i in range(len(df))]

In [27]:
df.to_csv(path_corrected_capa_accueil)

### Contraintes réseau 

Par exemple pour Rhone Alpes :

https://www.contraintes-reseau-s3renr-rte.com/region/auvergne-rhône-alpes

In [21]:
path_contraintes_ARA = '/Users/matthiasmolenat/repos/congestion/data/contraintes_ARA.csv'
df_contraintes_ARA = pd.read_csv(path_contraintes_ARA, delimiter=';')

In [9]:
df_contraintes_ARA.head()

,,,,,Donnees de la region
Puissance ENR installee,Puissance Totale a compenser,Energie non-evacuee moyenne Printemps,Energie non-evacuee moyenne Ete,Energie non-evacuee moyenne Automne,Energie non-evacuee moyenne Hiver
13251MW,83.00MW,23.00MWh,1049.00MWh,14.00MWh,0.00MWh
Donnees de zones de la region,NaN,NaN,NaN,NaN,NaN
Zone,Puissance totale a compenser,Nombre de contraintes,Energie non-evacuee moyenne,NaN,NaN
Ain,6.00,3,526MWh/an,NaN,NaN
